In [2]:
from llama_index.llms.ollama import Ollama
import pandas as pd
from jinja2 import Template
import re
llm = Ollama(model="llama3.1:latest", request_timeout=120.0, json_mode=True, use_gpu=True)

In [3]:
prompt_template = Template(
    """
    <|begin_of_text|>

    <|start_header_id|>user<|end_header_id|>

    Document : {{document}}

    What is the publication date? Output as a structured JSON object with format DD/MM/YYYY.
    <|eot_id|>

    {'publication_date':
    """
)

response = llm.complete(
    """
    CONSEIL MUNICIPAL DU 16 FEVRIER 2023
COMPTE RENDU

Le premier décembre deux mille vingt-deux, les membres du Conseil municipal, convoqués par Mme la
Maire le vingt-deux novembre deux mille vingt-deux, se sont réunis en séance publique, à l'Hôtel de Ville, à
20h30 sous la présidence de Mme la Maire.

Etaient présents :
Mme Hélène DE COMARMOND, Maire
    What is the publication date? Output as a structured JSON object with format DD/MM/YYYY."""
)
print(str(response))

{"publicationDate": "16/02/2023"}


In [4]:
df = pd.read_pickle('df_with_regex_chunks.pkl')
df.sample(5)

,doc_id,url,cache,text version,nature,published,entity,entity_type,Text,Gold published date,regex_chunks,regex_chunks_strict
388,1500/d288bade3937353cb39a434a7282ad7e1bb824e5_...,https://www.chelles.fr/wp-content/uploads/2022...,https://datapolitics-public.s3.gra.io.cloud.ov...,https://datapolitics-public.s3.gra.io.cloud.ov...,pv.cr,17/11/2020,Chelles,Commune,Direction de la Communication\n\nRÉPUBLIQUE FR...,17/11/2020,"[SEANCE DU 17 NOVEMBRE 2020, Le mardi 17 novem...","[SEANCE DU 17 NOVEMBRE 2020, Le mardi 17 novem..."
91,6357/8bef3_1698228857-PV---Conseil-Municipal-0...,http://www.ville-saint-ay.fr/userfile/fichier-...,https://datapolitics-public.s3.gra.io.cloud.ov...,https://datapolitics-public.s3.gra.io.cloud.ov...,pv.cr,06/02/2023,Saint-Ay,Commune,Page: page_1\nContent: \n\nPROCES-VERBAL DE L...,06/02/2023,"[LUNDI 6 FEVRIER 2023, le 6 février 2023, s’es...","[le 6 février 2023, s’est réuni en session ord..."
477,2529/075715514646513a108970a91ac4ed34a523bbed_...,https://www.huisne-sarthoise.com/wp-content/up...,https://datapolitics-public.s3.gra.io.cloud.ov...,https://datapolitics-public.s3.gra.io.cloud.ov...,pv.cr,29/01/2024,CC du Pays de l'Huisne Sarthoise,Intercommunalité,CONSEIL COMMUNAUTAIRE \nLundi 29 Janvier 2024 ...,29/01/2024,"[D 202, Lundi 29 Janvier 2024 à 18 H 00, L'an ...","[Le Conseil adopte, à l'unanimité, le procès-v..."
55,1749/bc84b_PV-CM-2-fevrier-2023-VF-1.pdf,https://www.suresnes.fr/wp-content/uploads/202...,https://datapolitics-public.s3.gra.io.cloud.ov...,https://datapolitics-public.s3.gra.io.cloud.ov...,pv.full,02/02/2023,Suresnes,Commune,VILLE DE SURESNES\nConseil Municipal\nSéance d...,02/02/2023,"[Séance du 2 février 2023, Page 1 sur 88, Appr...","[Séance du 2 février 2023, Approbation du proc..."
491,3897/75334104b076f7d0ea682cfbbb39249b2d70ccc7_...,https://www.gennesvaldeloire.fr/medias/2024/03...,https://datapolitics-public.s3.gra.io.cloud.ov...,https://datapolitics-public.s3.gra.io.cloud.ov...,pv.cr,18/12/2023,Gennes-Val-de-Loire,Commune,Département\r\nMaine-et-Loire\r\nArrondissemen...,26/12/2023,"[Séance du 18 décembre 2023, 18 décembre, 19 h...","[Séance du 18 décembre 2023, 12 décembre 2023,..."


In [5]:
def predict_date(document, temperature=0.5):
    input = {'document': ' '.join(document)}
    prompt = prompt_template.render(**input)
    output = llm.complete(prompt, max_tokens = 25, temperature=temperature)
    date = re.findall(r"\d{2}/\d{2}/\d{4}", str(output))
    if date: return date[0]
    else: return str(output)

In [6]:
test = df.sample(10)
print(test['published'])
test['prediction'] = test.apply(lambda x: predict_date(x['regex_chunks']), axis=1)
print(test['prediction'])

79     15/03/2023
493    06/02/2024
306    11/05/2022
478    19/06/2020
130    13/01/2023
259    30/06/2022
41     01/03/2023
116    26/01/2023
400    29/09/2020
212    23/06/2022
Name: published, dtype: object
79                                      15/03/2023
493                                     31/01/2024
306                                     18/12/2017
478                                     19/06/2020
130                                     13/01/2023
259    {  \n  "publication_date": "DD/MM/YYYY" \n}
41         { \n  "publication_date": "Unknown" \n}
116                                     25/01/2023
400                                     29/09/2020
212                                     23/06/2022
Name: prediction, dtype: object


In [7]:
df['prediction'] = df.apply(lambda x: predict_date(x['regex_chunks']), axis=1)

In [12]:
df[['published','prediction']]

,published,prediction
0,16/01/2023,16/01/2023
1,25/01/2023,25/01/2023
2,31/01/2023,02/02/2023
3,26/01/2023,26/01/2023
4,16/01/2023,16/01/2023
...,...,...
495,24/01/2024,24/01/2024
496,09/01/2024,09/01/2024
497,22/11/2022,22/11/2022
498,21/12/2023,21/12/2023


In [13]:
accuracy = (df["published"] == df["prediction"]).mean() * 100
print(accuracy)

61.4


In [10]:
df.to_pickle('ollama.pkl')

In [18]:
pattern = r"^\d{2}/\d{2}/\d{4}$"
print(len(df))
non_empty = df[df['regex_chunks'] != '']
print(len(non_empty))
non_empty = df[df['prediction'].str.match(pattern)]
print(len(non_empty))
accuracy = (non_empty["published"] == non_empty["prediction"]).mean() * 100
print(accuracy)

500
473
455
67.47252747252747
